In [46]:
import numpy as np
import re
import random
from solver import vertex_cover

In [47]:
def getNodes(graph):
    nodes = []
    for n in graph.keys():
        nodes.append(n)
    for n2 in graph.values():
        if type(n2) == "string":
            nodes.append(n2)
        else:
            nodes += n2
    return np.unique(nodes).tolist()
    

def getEdges(graph, isDirected):
    edges = []
    for item in graph.items():
        for destination in item[1]:
            if (not isDirected):
                string1 = item[0] if item[0] < destination else destination
                string2 = destination if item[0] < destination else item[0]
            else:
                string1 = item[0]
                string2 = destination
            edges.append(string1 + string2)
    return np.unique(edges).tolist()

def getEdgesPair(graph, isDirected):
    edges = []
    for item in graph.items():
        for destination in item[1]:
            if (not isDirected):
                string1 = item[0] if item[0] < destination else destination
                string2 = destination if item[0] < destination else item[0]
            else:
                string1 = item[0]
                string2 = destination
            edges.append((string1,string2))
    return edges

In [48]:
graph0 = {
    "a": ["b"]
}
#A---B

graph1 = {
    "a": ["b"],
    "b": ["c"]
}
#A----B----C   D=v0

graph2 = {
    "a": ["b", "c", "d"],
    "b": ["a"],
    "c": ["a"],
    "d": ["a"]
}

#       C          D
#        \        /
#         \      /
#          \    /
#           \  /
# B----------A

graph3 = {
    "a": "b",
    "b": "c",
    "c": "d"
}
# A--B--C--D

graph4 = {
    "a": "b",
    "b": "c",
    "c": "d",
    "d": "e"
}
# A--B--C--D--E

graph5 = {
    "a": ["b", "c", "d", "e"]
}
#       C          D
#        \        /
#         \      /
#          \    /
#           \  /
# B----------A-------------E

graph6 = {
    "a": ["b", "c", "d", "e"],
    "d": ["e"],
    "c": ["b", "e"]
}
#       C           D
#      / \____    /  \
#     /   \   \__/    \
#    /     \    /\_____\
#   /       \  /       \\
# B----------A-----------E


In [49]:
class Player:
    def __init__(self, types, shorthand, roundNum, isCover=False):
        self.types = types
        self.shorthand = shorthand
        self.roundNum = roundNum
        self.isCover = isCover
    def self_introduce(self):
        return f'Player {self.shorthand} for round {self.roundNum}'

def buildPlayers(nodes, edges, covers):
    # objectives
    objectivelayer = [Player('objective', 'o', 1)]
    # nodes
    n = len(nodes)
    vertexplayers = [Player('vertex', 'v-0', 1)]
    for i in nodes:
        vertexplayers.append(Player('vertex', f'v-{i}', 1, i in covers))
    # edges
    m = len(edges)
    edgeplayers = []
    for j in edges:
        edgeplayers.append(Player('edge', f'e-{j}', 1))
    # filler    
    k = len(covers)
    roundLimit1 = int(np.ceil(np.log(n-k)))
    fillerplayers = []
    for r in range(1, roundLimit1+1):
        for i in range(1,k+1):
            #for j in vertexplayers:
            fillerplayers.append(Player('filler', f'f^{r}_v#{i}', r))
    roundLimit2 = int(np.ceil(np.log(n-k)) + np.ceil(np.log(m)))
    for r in range(roundLimit1+1, roundLimit2+1):
        for i in range(1,k+1):
            #for j in edgeplayers:
            fillerplayers.append(Player('filler', f'f^{r}_e#{i}', r))
    # holder
    holderplayers = []
    for ei in edgeplayers:
        for l in range(0, 2**roundLimit1 - 1):
            holderplayers.append(Player('holder', f'h^{l}_{ei.shorthand}', 1))
            
    for fi in fillerplayers:
        fillerplayerR = fi.roundNum
        for l in range(0, 2**(fillerplayerR -1) -1):
            holderplayers.append(Player('holder', f'h^{l}_{fi.shorthand}', r))
    kfiller = 2**(roundLimit2 + int(np.ceil(np.log(k+1)))+1) - 1
    for l in range(0, kfiller):
        holderplayers.append(Player('holder', f'h^{l}_o', 1))
    return {
        'objectivePlayer': objectivelayer,
        'vertexPlayers': vertexplayers, 
        'edgePlayers': edgeplayers,
        'fillerPlayers': fillerplayers,
        'holderPlayers': holderplayers
    }

In [53]:
def pair(p1, p2, r1, r2, matches):
    matches.append((p1, p2))
    if p1 in r1:
        r1.remove(p1)
    if p2 in r2:
        r2.remove(p2)
    
def matchingPhase1(playerList, r):
    seeds = []
    objective = playerList['objectivePlayer'][0]
    holderString = playerList['holderPlayers']
    remainingHolders = holderString.copy()
    
    vertexString = playerList['vertexPlayers']
    remainingVertices = vertexString.copy()
    
    edgeString = playerList['edgePlayers']
    remainingEdges = edgeString.copy()
    
    fillerString = playerList['fillerPlayers']
    remainingFillers = fillerString.copy()
    
    matchedPlayers = []
    for holderPlayer in holderString:
        if 'o' in holderPlayer.shorthand:
            seeds.append((objective, holderPlayer))
            remainingHolders.remove(holderPlayer)
            break
    holderString = remainingHolders.copy()
    for vertexPlayer in vertexString:
        if vertexPlayer.isCover:
            for fillerPlayer in fillerString:
                if (
                        "v" in fillerPlayer.shorthand and 
                        fillerPlayer.roundNum == r and 
                        fillerPlayer not in matchedPlayers and 
                        vertexPlayer not in matchedPlayers
                    ):
                    pair(fillerPlayer, vertexPlayer, remainingFillers, remainingVertices, seeds)
                    matchedPlayers.append(fillerPlayer)
                    matchedPlayers.append(vertexPlayer)
        else:
            for vertexPlayer2 in remainingVertices:
                matchCountvertexPlayer2 = 0
                if (
                        vertexPlayer.shorthand != vertexPlayer2.shorthand and 
                        not vertexPlayer2.isCover and 
                        vertexPlayer2 not in matchedPlayers and 
                        vertexPlayer not in matchedPlayers
                    ):
                    pair(vertexPlayer, vertexPlayer2, remainingVertices, remainingVertices, seeds)
                    matchedPlayers.append(vertexPlayer)
                    matchedPlayers.append(vertexPlayer2)
    fillerString = remainingFillers.copy()
    vertexString = remainingVertices.copy()
    
    for edgePlayer in edgeString:
        for holdPlayer in holderString:
            if (
                    edgePlayer.shorthand in holdPlayer.shorthand and 
                    edgePlayer not in matchedPlayers and 
                    holdPlayer not in matchedPlayers
                ):
                pair(edgePlayer, holdPlayer, remainingEdges, remainingHolders, seeds)
                matchedPlayers.append(edgePlayer)
                matchedPlayers.append(holdPlayer)
    holderString = remainingHolders.copy()
    edgeString = remainingEdges.copy()
    
    for fillerPlayer in fillerString:
        for holdPlayer in holderString:
            if (
                    fillerPlayer.shorthand in holdPlayer.shorthand and 
                    fillerPlayer not in matchedPlayers and 
                    holdPlayer not in matchedPlayers
                ):
                pair(fillerPlayer, holdPlayer, remainingFillers, remainingHolders, seeds)
                matchedPlayers.append(fillerPlayer)
                matchedPlayers.append(holdPlayer)
    holderString = remainingHolders.copy()
    fillerString = remainingFillers.copy()
    
    for holderPlayer in holderString:
        for holderPlayer2 in remainingHolders:
            holderPlayerName = holderPlayer.shorthand
            holderPlayer2Name = holderPlayer2.shorthand
            if (
                    'f' in holderPlayerName and 
                    'f' in holderPlayer2Name and 
                    holderPlayerName != holderPlayer2Name and 
                    list(holderPlayerName)[3:] == list(holderPlayer2Name)[3:] and
                    holderPlayer not in matchedPlayers and
                    holderPlayer2 not in matchedPlayers
                ):
                pair(holderPlayer, holderPlayer2, remainingHolders, remainingHolders, seeds)
                matchedPlayers.append(holderPlayer)
                matchedPlayers.append(holderPlayer2)
                
            if (
                    'e' in holderPlayerName and 
                    'e' in holderPlayer2Name and 
                    holderPlayerName != holderPlayer2Name and 
                    list(holderPlayerName)[3:] == list(holderPlayer2Name)[3:] and
                    holderPlayer not in matchedPlayers and
                    holderPlayer2 not in matchedPlayers
                ):
                pair(holderPlayer, holderPlayer2, remainingHolders, remainingHolders, seeds)
                matchedPlayers.append(holderPlayer)
                matchedPlayers.append(holderPlayer2)
    holderString = remainingHolders.copy()
    #straggers:
    remainings = remainingFillers + remainingHolders + remainingEdges + remainingVertices;
    while len(remainings) >= 2:
        selectPairs = random.sample(remainings, 2)
        pair(selectPairs[0], selectPairs[1], remainings, remainings, seeds)
#     print('seeds:', len(seeds))
#     print('remainings:',len(remainings))
    return seeds

def decider(player1, player2):
    #print(f'{player1.self_introduce()} vs. {player2.self_introduce()}')
    type1 = player1.types
    type2 = player2.types
    if type1 == 'objective':
        if type2 == 'vertex':
            return player1
        elif type2 == 'edge':
            return player2
        elif type2 == 'filler':
            return player2
        elif type2 == 'holder':
            if 'o' in player2.shorthand:
                return player1
            else:
                return player2
            
    elif type1 == 'vertex':
        if type2 == 'vertex':
            if player1.shorthand < player2.shorthand:
                return player1
            else:
                return player2
        elif type2 == 'edge':
            vertice = re.search(r"v-(\w+)", player1.shorthand).group(1)
            edge = re.search(r"e-(\w+)", player2.shorthand).group(1)
            if vertice in edge:
                return player1
            else:
                return player2
        elif type2 == 'filler':
            return player1
        elif type2 == 'holder':
            return player2
        elif type2 == 'objective':
            return player2
        
    elif type1 == 'edge':
        if type2 == 'vertex':
            vertice = re.search(r"v-(\w+)", player1.shorthand).group(1)
            edge = re.search(r"e-(\w+)", player2.shorthand).group(1)
            if vertice in edge:
                return player2
            else:
                return player1
        elif type2 == 'edge':
            return random.choice([player1, player2])
        elif type2 == 'filler':
            return random.choice([player1, player2])
        elif type2 == 'holder':
            if 'e-' in player2.shorthand:
                if player1.shorthand in player2.shorthand:
                    return player1
                else:
                    return random.choice([player1, player2])
            else:
                return player2
        elif type2 == 'objective':
            return player1
    
    elif type1 == 'filler':
        if type2 == 'vertex':
            return player2
        elif type2 == 'edge':
            return random.choice([player1, player2])
        elif type2 == 'filler':
            return random.choice([player1, player2])
        elif type2 == 'holder':
            if 'e-' in player2.shorthand:
                return random.choice([player1, player2])
            elif  'o' in player2.shorthand or  player1.shorthand in player2.shorthand:
                return player1
            else:
                return random.choice([player1, player2])
        elif type2 == 'objective':
            return player1

    elif type1 == 'holder':
        if 'e-' in player1.shorthand:
            if type2 == 'vertex':
                return player1
            elif type2 == 'edge':
                if player2.shorthand in player1.shorthand:
                    return player2
                else:
                    return random.choice([player1, player2])
            elif type2 == 'filler':
                return random.choice([player1, player2])
            elif type2 == 'holder':
                if 'o' in player2.shorthand:
                    return player1
                else:
                    return random.choice([player1, player2])
            elif type2 == 'objective':
                return player1
        elif 'f^' in player1.shorthand:
            if type2 == 'vertex':
                return player1
            elif type2 == 'edge':
                return player2
            elif type2 == 'filler':
                if player2.shorthand in player1.shorthand:
                    return player2
                else:
                    return random.choice([player1, player2])
            elif type2 == 'holder':
                if 'o' in player2.shorthand:
                    return player1
                else:
                    return random.choice([player1, player2])
            elif type2 == 'objective':
                return player1
        elif 'o' in player1.shorthand:
            if type2 == 'vertex':
                return player1
            elif type2 == 'edge':
                return player2
            elif type2 == 'filler':
                return player2
            elif type2 == 'holder':
                if 'o' in player2.shorthand:
                    return random.choice([player1, player2])
                else:
                    return player2
            elif type2 == 'objective':
                return player2
                
def getRoundResult(r):
    result = {
        'objectivePlayer': [],
        'vertexPlayers': [], 
        'edgePlayers': [],
        'fillerPlayers': [],
        'holderPlayers': []
    }
    for match in r:
        matchResult = decider(match[0], match[1])
        if matchResult.types == 'vertex':
            result['vertexPlayers'].append(matchResult)
        elif matchResult.types == 'edge':
            result['edgePlayers'].append(matchResult)
        elif matchResult.types == 'filler':
            result['fillerPlayers'].append(matchResult)
        elif matchResult.types == 'objective':
            result['objectivePlayer'].append(matchResult)
        elif matchResult.types == 'holder':
            result['holderPlayers'].append(matchResult)
    return result

def matchingPhase2(playerList, r, nodes, edges, covers):
    seeds = []
    objective = playerList['objectivePlayer'][0]
    holderString = playerList['holderPlayers']
    remainingHolders = holderString.copy()
    
    vertexString = playerList['vertexPlayers']
    remainingVertices = vertexString.copy()
    
    edgeString = playerList['edgePlayers']
    remainingEdges = edgeString.copy()
    
    fillerString = playerList['fillerPlayers']
    remainingFillers = fillerString.copy()
    
    matchedPlayers = []
    for vertexPlayer in vertexString:
        for edgePlayer in edgeString:
            vertice = re.search(r"v-(\w+)", vertexPlayer.shorthand).group(1)
            edge = re.search(r"e-(\w+)", edgePlayer.shorthand).group(1)
            if vertice in edge and not vertexPlayer in matchedPlayers and not edgePlayer in matchedPlayers:
                pair(vertexPlayer, edgePlayer, remainingVertices, remainingEdges, seeds)
                matchedPlayers.append(vertexPlayer)
                matchedPlayers.append(edgePlayer)
    vertexString = remainingVertices.copy()
    edgeString = remainingEdges.copy()
    #pair remaining vertex with a filler
    for vertexPlayer in vertexString:
        for fillerPlayer in fillerString:
            if not vertexPlayer in matchedPlayers and not fillerPlayer in matchedPlayers:
                pair(vertexPlayer, fillerPlayer, remainingVertices, remainingFillers, seeds)
                matchedPlayers.append(vertexPlayer)
                matchedPlayers.append(fillerPlayer)
    vertexString = remainingVertices.copy()
    fillerString = remainingFillers.copy()
    
    for edgePlayer in edgeString:
        for edgePlayer2 in remainingEdges:
            if checkStringShare(edgePlayer.shorthand, edgePlayer2.shorthand) and edgePlayer.shorthand != edgePlayer2.shorthand:
                pair(edgePlayer, edgePlayer2, remainingEdges, remainingEdges, seeds)
                matchedPlayers.append(edgePlayer)
                matchedPlayers.append(edgePlayer2)
    edgeString = remainingEdges.copy()
    
    for fillerPlayer in fillerString:
        for holderPlayer in holderString:
            if (
                    fillerPlayer.shorthand in holderPlayer.shorthand and
                    fillerPlayer not in matchedPlayers and
                    holderPlayer not in matchedPlayers
                ):
                pair(fillerPlayer, holderPlayer, remainingFillers, remainingHolders, seeds)
                matchedPlayers.append(fillerPlayer)
                matchedPlayers.append(holderPlayer)
    fillerString = remainingFillers.copy()
    holderString = remainingHolders.copy()
    
    #pair remaining edges
    for edgePlayer in edgeString:
        for fillerPlayer in fillerString:
            if (
                    'e' in fillerPlayer.shorthand and
                    edgePlayer not in matchedPlayers and
                    fillerPlayer not in matchedPlayers
                ):
                pair(edgePlayer, fillerPlayer, remainingEdges, remainingFillers, seeds)
                matchedPlayers.append(edgePlayer)
                matchedPlayers.append(fillerPlayer)
    edgeString = remainingEdges.copy()
    fillerString = remainingFillers.copy()

    for holderPlayer in holderString:
        if 'o' in holderPlayer.shorthand:
            seeds.append((objective, holderPlayer))
            remainingHolders.remove(holderPlayer)
            break
    holderString = remainingHolders.copy()
    #straggers:
    remainings = remainingFillers + remainingHolders + remainingEdges + remainingVertices;
    for p1 in remainings:
        for p2 in remainings:
            if (p1.types == "edge"):
                if (
                        p2.types == 'filler' and 'e' in p2.shorthand and
                        p1 not in matchedPlayers and
                        p2 not in matchedPlayers
                    ):
                    pair(p1,p2, remainings, remainings, seeds)
                    matchedPlayers.append(p1)
                    matchedPlayers.append(p2)
    while len(remainings) >= 2:
        selectPairs = random.sample(remainings, 2)
        pair(selectPairs[0], selectPairs[1], remainings, remainings, seeds)
    return seeds

def matchingFinal(playerList):
    objective = playerList['objectivePlayer']
    vertexString = playerList['vertexPlayers']
    holderString = playerList['holderPlayers']
    seeds = []
    remainings = objective + vertexString + holderString
    while len(remainings) >= 2:
        selectPairs = random.sample(remainings, 2)
        pair(selectPairs[0], selectPairs[1], remainings, remainings, seeds)
    return seeds

def bracketMaker(p, nodes, edges, covers):
    #phase1
    n = len(nodes)
    m = len(edges)
    k = len(covers)
    pri = p
    playerNums = int(np.sum([
        len(p['objectivePlayer']),
        len(p['vertexPlayers']), 
        len(p['edgePlayers']),
        len(p['fillerPlayers']),
        len(p['holderPlayers'])
    ]))
    bracket = {}
    for i in range(1, int(np.ceil(np.log(n-k)))+1):
        ri = matchingPhase1(pri, i)
        pri = getRoundResult(ri)
        bracket[f'round{i}'] = ri
    #phase2
    for j in range(int(np.ceil(np.log(n-k)))+1,int(np.ceil(np.log(m)) + np.ceil(np.log(k+1)) + 1 +1)):
        rj = matchingPhase2(pri, j, nodes, edges, covers)
        pri = getRoundResult(rj)
        bracket[f'round{j}'] = rj
    #final
    for f in range(int(np.ceil(np.log(m)) + np.ceil(np.log(k+1)) + 1 +1), int(np.log2(playerNums) + 1)):
        rf = matchingFinal(pri)
        pri = getRoundResult(rf)
        bracket[f'round{f}'] = rf
    return bracket

def checkStringShare(str1, str2):
    joinString = str1 + str2
    charArray = list(joinString)
    newArray = np.unique(charArray)
    if len(newArray) < len(charArray):
        return True
    return False

In [54]:
def bracketGenerator(graph):
    chosenGraph = graph
    nodes = getNodes(chosenGraph)
    edges = getEdges(chosenGraph, False)
    edgesPair = getEdgesPair(chosenGraph, False)
    covers = vertex_cover(nodes, edgesPair) #put solver
    players = buildPlayers(nodes, edges, covers)
    #rounds of number of players
    def getFullPlayers(playerNums):
        n = 0
        stopFlag = False
        while (not stopFlag):
            if playerNums > 2**n:
                n += 1
            else:
                stopFlag = True
        return 2**n
    def fillPlayers(playersList):
        playerNums = np.sum([
            len(playersList['objectivePlayer']),
            len(playersList['vertexPlayers']), 
            len(playersList['edgePlayers']),
            len(playersList['fillerPlayers']),
            len(playersList['holderPlayers'])
        ])
        if getFullPlayers(playerNums) > playerNums:
            for i in range(0, getFullPlayers(playerNums) - playerNums):
                players['holderPlayers'].append(Player('holder', f'h^{i}_o_ammend', 1))
        return playersList
    players = fillPlayers(players)
    tour = bracketMaker(players, nodes, edges, covers)
    return tour

In [55]:
tourTest = bracketGenerator(graph6)

# print('----------------------------')
# for i in tour['round2']:
#     print(f'{i[0].self_introduce()} vs. {i[1].self_introduce()}')

# r2_remains = getRoundResult(tour['round2'])

# # playnum = 0
# for i in getRoundResult(tour['round2']).values():
#     for p in i:
#         print(p.self_introduce())
# # print(playnum)
# r3 = matchingPhase2(r2_remains, 3, graph1)
# for i in r3:
#     print(f'{i[0].self_introduce()} vs. {i[1].self_introduce()}')
    
# for j in getRoundResult(r3).values():
#     for p in j:
#         print(p.self_introduce())

['a', 'b', 'c', 'd', 'e']
[('a', 'b'), ('a', 'c'), ('a', 'd'), ('a', 'e'), ('d', 'e'), ('b', 'c'), ('c', 'e')]
[[['a', 'b'], [1, 1]], [['a', 'c'], [1, 1]], [['a', 'd'], [1, 1]], [['a', 'e'], [1, 1]], [['d', 'e'], [1, 1]], [['b', 'c'], [1, 1]], [['c', 'e'], [1, 1]]]
['ab', 'ac', 'ad', 'ae', 'de', 'bc', 'ce']
[1, 1, 1, 1, 1, 1, 1]
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 5.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 0 columns.
MIP Presolve modified 3 coefficients.
Reduced MIP has 3 rows, 5 columns, and 9 nonzeros.
Reduced MIP has 5 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 3 rows, 5 columns, and 9 nonzeros.
Reduced MIP has 5 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 

In [62]:
for i in tourTest['round4']:
    print(f'{i[0].self_introduce()} vs. {i[1].self_introduce()}')

Player v-a for round 1 vs. Player f^3_e#3 for round 3
Player v-e for round 1 vs. Player f^2_e#2 for round 2
Player v-b for round 1 vs. Player f^2_e#3 for round 2
Player o for round 1 vs. Player h^17_o for round 1
Player h^14_o for round 1 vs. Player h^16_o_ammend for round 1
Player v-0 for round 1 vs. Player h^31_o for round 1
Player h^3_o for round 1 vs. Player h^22_o_ammend for round 1
Player h^9_o_ammend for round 1 vs. Player h^34_o for round 1


In [ ]:
# idea comes from this very very old paper
# https://www.barton.edu/pdf/faculty-publications/bengtson-winning-probabilities-publication.pdf

import numpy as np

p = [[1, 0.434, 0.337, 0.298, 0.262, 0.277, 0.312, 0.485],
[0.566, 1, 0.571, 0.503, 0.42, 0.456, 0.516, 0.546],
[0.663, 0.429, 1, 0.432, 0.356, 0.389, 0.463, 0.424],
[0.702, 0.497, 0.568, 1, 0.416, 0.454, 0.514, 0.492],
[0.738, 0.58, 0.644, 0.584, 1, 0.592, 0.596, 0.576],
[0.723, 0.544, 0.611, 0.546, 0.408, 1, 0.56, 0.538],
[0.688, 0.484, 0.537, 0.486, 0.404, 0.44, 1, 0.478],
[0.515, 0.454, 0.576, 0.508, 0.424, 0.462, 0.522, 1]]



# bracket kingstone is making
tournament = [[(0,1),(2,3),(4,5),(6,7)],#round 1 
             [(0,2),(4,6)],#round 2
             [(0,4)]]#round3

#gives probability of winnig i agains j (philip func)
def P(i,j):
    return p[i][j]
 
#just a function can go through braket and find the opponent
def opponent(player_i,round):
    for (i,j) in tournament[round-1]:
        if i == player_i:
            return j
        elif j == player_i:
            return i
    return None

#go to the refrence for this one
def Win_rate(player_i,round):
    if round == 0 : return 1
    opp_player = opponent(player_i,round)
    if opp_player == None: return 0
    return Win_rate(player_i,round-1) * P(player_i,opp_player) * Win_rate(opp_player,round-1)

Win_rate(2,3)